In [ ]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

connection = sqlite3.connect(os.getenv("SQLITE_DB_FILE"))
cursor = connection.cursor()
query = """
SELECT activity.timestamp, video.title, video.length, channel.name AS channel_name, category.name AS category_name
FROM activity
JOIN video ON activity.video_id = video.id
JOIN channel ON activity.channel_id = channel.id
JOIN category ON channel.category_id = category.id
"""

df = pd.read_sql_query(query, connection)
connection.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_month'] = df['timestamp'].dt.strftime('%Y-%m')

df_pivot = df.pivot_table(columns='year_month', index='category_name', aggfunc='size').fillna(0)
#df_pivot = df_pivot.drop(['Education and Learning', 'News and Politics', 'Documentaries and Movies'], axis=0)

category_totals = df_pivot.sum(axis=1)
sorted_categories = category_totals.sort_values(ascending=False).index
top_categories = sorted_categories[:7]
df_pivot_top = df_pivot.loc[top_categories]

# Customize x-axis labels
def format_tick(tick_val, tick_pos):
    tick_label = df_pivot_top.columns[int(tick_val)]
    if '-01' in tick_label:
        return tick_label[:4]
    else:
        return ''
    
# Calculate the rolling average
window_size = 4
df_pivot_top_smooth = df_pivot_top.rolling(window=window_size, axis=1, center=True).mean()

plt.style.use('dark_background')
plt.figure(figsize=(12, 3))
ax = sns.heatmap(df_pivot_top_smooth, alpha=0.8, cmap='YlGnBu_r', annot=False, fmt='g', linewidths=0)

ax.xaxis.set_major_formatter(FuncFormatter(format_tick))
ax.set_xticks(range(len(df_pivot_top.columns)))
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.title('Number of Watches per Month (Smoothed)', fontsize=10)
plt.xlabel('')
plt.ylabel('')
ax.invert_yaxis()
plt.show()